In [36]:
import os
import sys
import subprocess
import conda
import importlib
import socket
import getpass
import datetime

run = lambda *cmd: subprocess.run(cmd, capture_output=True)

### Helper Functions

In [40]:
def check_version(name, failed=False):
    version = run(name,'--version')
    if version.stderr:
        install = run('conda','install','-y',name)
        return install
    else:
        return version

def check_import(name, failed=False):
    try:
        m = importlib.import_module(name)
        res = m.__version__
    except ImportError as e:
        if not failed:
            run('pip','install',name)
            return check_import(name, failed=True)
        else:
            res = repr(e)
    return res

## Check conda path

The installation path of anaconda cannot contain any space characters (```' '```).

In [41]:
conda_path_ok = conda.CONDA_PACKAGE_ROOT.find(' ') < 0

In [64]:
conda = run('conda','info')

In [65]:
conda_version = run('conda','--version')

In [79]:
# check version
if int(''.join(conda_version.stdout.decode('utf-8').strip()[-5:].split('.'))) < 400:
    conda_update = run('conda','update','-y','conda')
else:
    conda_update = None

In [ ]:
conda_update_all = run('conda','update','-y','--all')

## Check conda installs

In [49]:
python = check_version('python')
pip = check_version('pip')
git = check_version('git')
jupyter = check_version('jupyter')

In [58]:
%%capture vscode
# VS Code triggers FileNotFoundError when "run('code')"  on Windows
!code --version

## Check python packages

In [4]:
matplotlib = check_import('matplotlib')
numpy = check_import('numpy')
pandas = check_import('pandas')
pip_py = check_import('pip')
pytest = check_import('pytest')
hypothesis = check_import('hypothesis')

In [5]:
%%capture matplotlib_backend
%matplotlib

## Print Report

In [94]:
hostname = socket.gethostname()
username = getpass.getuser()
localtime = datetime.datetime.now().isoformat()

In [98]:
def write_report(*contents):
    with open("syscheck-report-{}-{}-{}.txt".format(hostname, username, datetime.datetime.now().date().isoformat()), 'w') as f:
        for line in contents:
            f.write(line + '\n')
        else:
            f.write(hash(reduce(lambda a,b: a+b, contents)))

SyntaxError: invalid syntax (<ipython-input-98-31af425b8583>, line 4)